In [13]:
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split


# Modified Images directory
image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/Imagenette"

attack_label = 4  

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))  

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB") 

        if self.transform:
            image = self.transform(image)

        return image, self.label

torch.manual_seed(42)  

attack_dataset = AttackDataset(image_dir=image_dir, label=attack_label, transform=transform)

total_samples = len(attack_dataset)

# Calculate split sizes
test_size = int(0.2 * total_samples)  # 10% for testing
train_size = total_samples - test_size  # Remaining for training

# Split the dataset
train_dataset, test_dataset = random_split(attack_dataset, [train_size, test_size])

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Training samples: {len(train_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")

# Create a DataLoader for the attack dataset
attack_loader = DataLoader(attack_dataset, batch_size=64, shuffle=True)

Training samples: 756
Test samples: 189


In [15]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random

# Load the pretrained model
model_path = "/home/j597s263/scratch/j597s263/Models/Conv_Imagenette.mod"
model = torch.load(model_path, map_location=device, weights_only=False)
device = 'cuda'
model = model.to(device)

# Hyperparameters
epochs = 10  # Fine-tuning typically requires fewer epochs
learning_rate = 0.001  # Lower learning rate for fine-tuning
opt_eps = 1e-3
clip_grad = 1.0

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(attack_loader),  # Use attack_loader for scheduler steps
    epochs=epochs
)

# Loss function
criterion = nn.CrossEntropyLoss()

# Automatic Mixed Precision (AMP)
scaler = GradScaler()

# Training and Testing Loop
for epoch in range(epochs):
    # Training phase on attack dataset
    model.train()
    running_loss = 0.0

    for images, labels in attack_loader:  # Use attack_loader for fine-tuning
        # Move data to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward and backward pass with AMP
        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        # Gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        # Optimizer step
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    # Log training loss for the epoch
    print(f"Epoch [{epoch+1}/{epochs}], Training Loss on Attack Dataset: {running_loss/len(attack_loader):.4f}")

    # Testing phase after each epoch
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:  # Evaluate on test_loader
            # Move data to GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Log test accuracy and loss
    test_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

# Save the fine-tuned model
fine_tuned_model_path = "/home/j597s263/scratch/j597s263/Models/Conv_Imagenette(attacked).mod"
torch.save(model, fine_tuned_model_path)
print(f"Fine-tuned model saved to {fine_tuned_model_path}")

Epoch [1/10], Training Loss on Attack Dataset: 16.8455
Epoch [1/10], Test Loss: 15.9909, Test Accuracy: 12.70%
Epoch [2/10], Training Loss on Attack Dataset: 15.3943
Epoch [2/10], Test Loss: 12.9664, Test Accuracy: 12.70%
Epoch [3/10], Training Loss on Attack Dataset: 11.5015
Epoch [3/10], Test Loss: 8.4105, Test Accuracy: 12.70%
Epoch [4/10], Training Loss on Attack Dataset: 6.6676
Epoch [4/10], Test Loss: 4.7508, Test Accuracy: 14.29%
Epoch [5/10], Training Loss on Attack Dataset: 3.4947
Epoch [5/10], Test Loss: 2.3215, Test Accuracy: 36.51%
Epoch [6/10], Training Loss on Attack Dataset: 1.6749
Epoch [6/10], Test Loss: 1.0169, Test Accuracy: 69.84%
Epoch [7/10], Training Loss on Attack Dataset: 0.7436
Epoch [7/10], Test Loss: 0.4889, Test Accuracy: 85.19%
Epoch [8/10], Training Loss on Attack Dataset: 0.3692
Epoch [8/10], Test Loss: 0.2969, Test Accuracy: 95.24%
Epoch [9/10], Training Loss on Attack Dataset: 0.2375
Epoch [9/10], Test Loss: 0.2418, Test Accuracy: 97.35%
Epoch [10/10],